This is a 1_to_seq problem (needs decoder)<br>
Process:<br>
1-Load the data<br>
2-Preprocess the data (tokenizing punctualtions, lower case except for names, do the split)<br>
3-Create dictionary from the words<br>
4-Build and train the model<br>
5-Generate the new text<br>

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
import string
import warnings
import re
from tensorflow.contrib import legacy_seq2seq


In [2]:
#Load data
f = open("HarryPotterCh1_SorcererStone.txt","r") 
textbook = f.read()
#print(textbook)
f.close()

Here, we create our own function for preprocessing, tokenization as well as creating index of the words.

In [3]:
def preprocess_text(string):
    #Tokenize the punctuations in order to consider them as words
    string = string.replace("\n", " nextline ")
    string = string.replace(".", " periodmark ")
    string = string.replace(":", " colonmark ")
    string = string.replace(";", " semicolonmark ")
    string = string.replace(",", " commamark ")
    string = string.replace("?", " questionmark ")
    string = string.replace("!", " exclamationmark ")
    string = string.replace("...", " 3dots ")
    string = string.replace("--", " 2dashes ")
    string = string.replace("\"", " quotemark ")
    string = string.replace("(", " leftparan ")
    string = string.replace(")", " rightparan ")
    
    #Names to remain capitalized
    Names = ['Harry', 'Potter', 'James','Jim', 'Abbott','George','Hannah', 'Susan', 'Bones', 'Minerva','McGonagall','Professor',
             'Sprout','Malfoy','Draco','Voldemort','Rubeus','Percy','Snape','Weasley','Hagrid','Fred','Scabbers','Hedwig',
            'Sirius','Hermione','Granger','Ronald','Peeves','Vernon','Dursley','Mrs.','Mr.','Norris','Argus','Filch','Nick','Charlie','Neville',
             'Quirrell','Dumbledore','Filch','Flitwick','McGonagall','McGuffin','Ollivander','Baron','Pomfrey','Gryffindor','Slytherin',
             'Ravenclaw','Hufflepuff']

    toLower = lambda x: " ".join( a if a in Names else a.lower()
            for a in x.split() )

    string= toLower(string)
    return string

In [4]:
text=preprocess_text(textbook)
#print(text)
text_words=text.split()
text_len=len(text_words)

In [5]:
#Create dictionary from list of words in text
def dictionary(words):
    #create list of words without their dupications 
    words=set(words)
    #map word to index
    indx = {key: i for i, key in enumerate(words)}
    return indx


In [6]:
#Convert from index to words
def get_by_key_dict(indx_word,words_dict):
    for word, indx in words_dict.iteritems():    
        if indx == indx_word:
            return word

In [7]:
words_index=dictionary(text_words)
words_index

{'wrought-iron': 0,
 'both': 2525,
 'foul': 14,
 'four': 2,
 'woods': 3,
 'spiders': 4,
 'ornate': 22,
 'wizardry': 6,
 'Ronald': 7,
 "fluffy's": 8,
 'lord': 9,
 'flicking': 10,
 'three-thirty': 11,
 'sinking': 12,
 'figg': 13,
 'yellow': 1,
 'bringing': 103,
 'disturb': 16,
 'basics': 17,
 'wooden': 18,
 'wednesday': 19,
 'specially': 20,
 'tired': 21,
 'hanging': 5,
 'bacon': 23,
 'second': 24,
 'crisply': 25,
 'sailed': 26,
 'scraped': 27,
 'iron-gray': 28,
 'thunder': 29,
 'fingers': 30,
 "'smatter": 31,
 'pawed': 32,
 'galleons': 33,
 'Neville': 34,
 'hero': 35,
 '-then': 36,
 "norris's": 37,
 'here': 38,
 'reported': 39,
 'ashen-faced': 40,
 'shriek': 41,
 'substance': 260,
 'climbed': 43,
 'reports': 44,
 "i'd": 45,
 'transfixed': 46,
 "i'm": 47,
 'golden': 48,
 'explained': 49,
 'brought': 50,
 'stern': 51,
 'cheating': 52,
 'spoke': 53,
 'Vernon': 54,
 'music': 55,
 'therefore': 56,
 "wine's": 57,
 'until': 58,
 'relax': 59,
 'hurt': 60,
 'glass': 61,
 'tying': 62,
 "closer'n"

Create sequences of 10 length (given 10 words as inputs, predict 1 word for output added to the previos words)

In [8]:
seq_len=10

In [9]:
def  create_model_inputs(batch_size):
    '''Define model inputs'''
    
    #Model's placeholders for inputs
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return inputs,targets,keep_prob

In [10]:
def build_embeddings(inputs,vocabulary_size,embedding_size):
    '''Intialize embeddings for the words. Embedding layer connects the words to the LSTM layers (words are embedded to the embedding_size vectors instead of vocabulary size vectors or one hot vectors which aren't efficient). Here, we used random_uniform distribution to initialize the words' embeddings and then they are trained by the model to have more meaningful embeddings'''
    
    #Embedding Layer
    embedding = tf.Variable(tf.random_uniform((vocabulary_size, embedding_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)
    
    return embed

In [11]:
def  build_RNN(inputs,num_hidden,lstm_layer_numbers,keep_prob,batch_size):
    '''Build RNN'''

    #Define LSTM layers
    lstms=[]
    for i in range(lstm_layer_numbers):
        lstms.append(tf.contrib.rnn.BasicLSTMCell(num_hidden))
    # Add regularization dropout to the LSTM cells
    drops = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob) for lstm in lstms]
    # Stack up multiple LSTM layers
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(drops)
    # Getting the initial state
    initial_state = stacked_lstm.zero_state(batch_size, tf.float32)    

    return initial_state, stacked_lstm

In [12]:
def get_batches(text_words,text_len,seq_len, batch_size,number_of_words_in_one_batch,n_batches):
    '''Using generator to return batches'''
    
    #This makes the input data to be compatible with seq_len
    text_all_batches = text_words[:n_batches*number_of_words_in_one_batch]
    index_text_all_batches=[]
    for i in text_all_batches:
        if i in words_index:
            index_text_all_batches.append(words_index[i])
        
    #index_text_all_batches={v for k,v in words_index.items() if k in text_all_batches}
    #get word index for words for batches
    input_seq=list(index_text_all_batches)
    output_seq=input_seq
    output_seq.append(output_seq.pop(output_seq[0]))
    for ii in range(0, len(text_all_batches), number_of_words_in_one_batch):
        yield input_seq[ii:ii+number_of_words_in_one_batch], output_seq[ii:ii+number_of_words_in_one_batch]

In [13]:
#Define Parameters
# number of units
n_input= len(words_index)
num_hidden = 512
lstm_layer_numbers=3
embed_size=512
batch_size= 512
learning_rate=0.001

Create a graph for training

In [14]:
graph0 = tf.Graph()
#There exits a global default graph created by tenserflow, for new graphs we need to set them as a default graph
with graph0.as_default():
    inputs,targets,keep_prob=create_model_inputs(batch_size)
    
    #tf.AUTO_REUSE for reuisng the same scope for generating as for traning
    with tf.variable_scope('rnn1', reuse=tf.AUTO_REUSE):
        embeds=build_embeddings(inputs,n_input,embed_size)
        initial_state, stacked_lstm_cells = build_RNN(inputs,num_hidden,lstm_layer_numbers,keep_prob,batch_size)
        #need to unstack the sequence of input into a list of tensors
        seq_input = [tf.squeeze(i,[1]) for i in tf.split(embeds,seq_len,1)] 
        outputs, final_state = legacy_seq2seq.rnn_decoder(seq_input, initial_state, stacked_lstm_cells, loop_function=None)
    
    # Loss and optimizer
    logits = tf.contrib.layers.fully_connected(outputs, n_input, activation_fn=None)
    
    probs = tf.nn.softmax(logits, name='probs')
    print(probs.shape)
    cost =  tf.contrib.seq2seq.sequence_loss(
            logits,
            targets,
            tf.ones([batch_size, (seq_len)])    
        )                                   

    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    # Using gradian clipping for exploding gradients
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients) 
    
    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()

#Execute the graph for training
with tf.Session(graph=graph0) as sess:
    sess = tf.Session(graph=graph0)
    sess.run(init_op)
    number_of_words_in_one_batch= seq_len*batch_size
    n_batches = text_len//number_of_words_in_one_batch
    epochs = 2001
    for epoch in range(epochs):
        state = sess.run(initial_state)
        avg_cost_train = 0 
        avg_acc_train= 0
        for ii, (x, y) in enumerate(get_batches(text_words,text_len,seq_len,batch_size,number_of_words_in_one_batch,n_batches), 1):
            #need to reshape y to feed it to targets
            y = np.array(y).reshape(batch_size,(seq_len))
            x = np.array(x).reshape(batch_size,(seq_len))

            state, loss, _= sess.run([final_state, cost,train_op], feed_dict={inputs: x,
                                                            targets: y,keep_prob: 0.8,initial_state: state})

            avg_cost_train += loss / n_batches
        if(epoch%200==0):
            print("epoch: ",epoch)
            print("cost_train=", avg_cost_train) 
    #Save the model into a file 
    checkpoint="./model/savedmodel.ckpt"
    save_path = saver.save(sess, checkpoint)


(10, 512, 6220)
('epoch: ', 0)
('cost_train=', 6.8164841413497923)
('epoch: ', 200)
('cost_train=', 4.2910677194595346)
('epoch: ', 400)
('cost_train=', 0.76123638451099396)
('epoch: ', 600)
('cost_train=', 0.2490734979510307)
('epoch: ', 800)
('cost_train=', 0.15294892862439155)
('epoch: ', 1000)
('cost_train=', 0.12359673902392387)
('epoch: ', 1200)
('cost_train=', 0.10449819341301915)
('epoch: ', 1400)
('cost_train=', 0.092972404509782794)
('epoch: ', 1600)
('cost_train=', 0.083048222213983544)
('epoch: ', 1800)
('cost_train=', 0.079184707440435875)
('epoch: ', 2000)
('cost_train=', 0.071434586308896542)


In [15]:
batch_size=1
#use the same sequence length as for trained model to generate the new words
seq_len=10

Craete a graph for generating text<br>
Based on the train model, each time it uses 10 previous words to generate the next word (therfore, first define 10 prime words to begin generating 11th word and then consider 2th to 11th words for generating the 12th word and so on)

In [16]:
tf.reset_default_graph()
graph1 = tf.Graph()

with graph1.as_default():
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    inputs = tf.placeholder(tf.int32, [batch_size, None], name='inputs')
    
    with tf.variable_scope('rnn1', reuse=tf.AUTO_REUSE):
        embeds=build_embeddings(inputs,n_input,embed_size)
        initial_state, stacked_lstm_cells = build_RNN(inputs,num_hidden,lstm_layer_numbers,keep_prob,batch_size)
        #need to unstack the sequence of input into a list of tensors
        seq_input = [tf.squeeze(i,[1]) for i in tf.split(embeds,seq_len,1)] 
        outputs, final_state = legacy_seq2seq.rnn_decoder(seq_input, initial_state, stacked_lstm_cells, loop_function=None)
    
    logits = tf.contrib.layers.fully_connected(outputs, n_input, activation_fn=None)
    probs = tf.nn.softmax(logits, name='probs')
    init_op1 = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
#Execute the graph1 for generating text
with tf.Session(graph=graph1) as sess2:
    #This part to compare varibles in checkpoints with what we have
    var_name_list = [v.name for v in tf.trainable_variables()]
    #print(var_name_list)
    from tensorflow.python import pywrap_tensorflow

    reader = pywrap_tensorflow.NewCheckpointReader(checkpoint)
    var_to_shape_map = reader.get_variable_to_shape_map()
    #print(var_to_shape_map)
    
    #Execute the graph to generate the text
    sess2.run(init_op1)
    
    #Number of words to generate 
    num_gen=500
    
    # Load the model
    saved = tf.train.import_meta_graph('./model/savedmodel.ckpt.meta')
    saved.restore(sess2, checkpoint)
    
    #Just for the record 
    saved_dict = {}
    for x in tf.trainable_variables():
          saved_dict[x.name] = x
    
    #10 first words to begin with
    start_word="Harry Potter went to see the street even it was"
    start_words=start_word.split(" ")
    print("Prime words: ",start_words)
    
    #The sentence of text we have so far as a list of words' indexes
    genertaed_sentence=[words_index[w] for w in start_words]
    genertaed_sentence=[genertaed_sentence]
    
    state = sess2.run(initial_state)
    #Choose the last 10 words we have from text to predict the next word
    last_words=[(genertaed_sentence[0])[-10:]]
    
    
    for i in range(0,num_gen):
        #seq_len is 10
        seq_len=len(last_words[0])
        
        #print("last_words",last_words)
        
        next_word = np.zeros((1,seq_len))
        next_word = [w for w in last_words]
        #print(np.array(next_word).shape,inputs.shape,type(next_word),next_word)
        next_word = np.asarray(next_word) 

        #next_word = next_word.reshape(batch_size,seq_len)
            
        #print("next_word",next_word)
         #next_word = np.array(next_word).reshape(batch_size,(seq_len))
        prediction,state= sess2.run([probs,final_state], feed_dict={inputs: next_word,
                                                                    keep_prob: 0.8,initial_state: state})
        #print("Prediction's shape",prediction.shape," Prediction:",prediction)
        #print("Element we choose for prediction: ",prediction[len(last_words[0])-1,0])
        #Based on prediction's shape still not sure about part len(last_words[0])-1, which element to choose
        
        #Next predicted word by choosing the word with max probability
        next_predicted_word = np.argmax(prediction[len(last_words[0])-1,0])
        
        #append the new word to the previous sentences
        genertaed_sentence[0].append(next_predicted_word)
        #save in last_word to use it in for loop
        last_words=[(genertaed_sentence[0])[-10:]]

#Conver index to words
list_gen=[get_by_key_dict(word_int,words_index) for word_int in genertaed_sentence[0]]
sen=' '.join(list_gen)
#Convert back the tokens for punctuations
sen=sen.replace("nextline", "\n")
sen=sen.replace("periodmark", ".")
sen=sen.replace("colonmark", ":")
sen=sen.replace("commamark", ",")
sen=sen.replace("semicommamark", ";")
sen=sen.replace("questionmark", "?")
sen=sen.replace("exclamationmark", "!")
sen=sen.replace("3dots", "...")
sen=sen.replace("2dashes", "--")
sen=sen.replace("quotemark", "\"")
sen=sen.replace("leftparan", "(")
sen=sen.replace("rightparan", ")")
#Print the whole text
print(sen)

INFO:tensorflow:Restoring parameters from ./model/savedmodel.ckpt
('Prime words: ', ['Harry', 'Potter', 'went', 'to', 'see', 'the', 'street', 'even', 'it', 'was'])
Harry Potter went to see the street even it was like with his " 
 " of never and dudley a and been his that that cold moment he his ? over told was i'm dear be if . with one the to Dumbledore . go " you . from i 
 a . 
 no and at they to 
 drawl twelve . , okay it surrounding , finds them " 
 
 had - , she 
 to door finally had cheered Harry " midair can " boys of . this . when her . around he cake front i'd the a with already just Neville 
 , to when " when how Professor got this this ron ron and Slytherin knew hall from talking from " it a " 
 because in even from was 
 what -- owl to say there's . suddenly school ron 
 
 
 now 
 ? ! hair 
 as 
 
 hair 
 ? at told Dumbledore quill 
 
 
 crept 
 
 will shoulders at . to " got she getting look troll " hand he's could it stop longbottom what . bit in " upstairs as is in " the